In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 

import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Load Dataset

In [ ]:
df = pd.read_csv('Data_Professional_Salary_Survey_Responses.csv')
df.rename(columns={' SalaryUSD ': 'SalaryUSD'}, inplace=True)
df

# Show the Count of each Columns 

In [ ]:
df.isna().sum()

# Data Cleaning:

#### Replacing missing values with np.nan

In [ ]:
missing_val = ['Not Asked']
df.replace(missing_val, np.nan, inplace = True)

#### Count of Nan's after replacing

In [ ]:
df.isna().sum()

#### Remove columns that contains NaN more than 30%

In [ ]:
df=df.drop(['PostalCode', 'HowManyCompanies', 'CompanyEmployeesOverall', 'Education', 'EducationIsComputerRelated', 
'Certifications', 'HoursWorkedPerWeek', 'TelecommuteDaysPerWeek', 'NewestVersionInProduction', 'OldestVersionInProduction',
'PopulationOfLargestCityWithin20Miles', 'OtherJobDuties', 'KindsOfTasksPerformed', 'LookingForAnotherJob'], axis = 1)

In [ ]:
df.isna().sum()

#### Fill nulls values

In [ ]:
df['OtherDatabases'] = df['OtherDatabases'].fillna(df['OtherDatabases'].mode()[0])
df['DatabaseServers'] = df['DatabaseServers'].fillna(df['DatabaseServers'].mode()[0])
df['CareerPlansThisYear'] = df['CareerPlansThisYear'].fillna(df['CareerPlansThisYear'].mode()[0])
# We can't fill the Gender by mode. We assume that it is Unknown
df['Gender'] = df['Gender'].fillna('Unknown')
df['Gender']= df['Gender'].replace(['None'],'Unknown')

#### Replace outliers by nulls

In [ ]:
print(df['YearsWithThisTypeOfJob'].unique())
df.loc[df['YearsWithThisTypeOfJob'] > 45]= np.nan

#### Fill the replaced values by the mean

In [ ]:
m= df['YearsWithThisTypeOfJob'].mean(skipna=True)
df['YearsWithThisTypeOfJob'] = df['YearsWithThisTypeOfJob'].fillna(round(m))

#### Convert to numeric

In [ ]:
#convert to numeric
df['SalaryUSD']=df["SalaryUSD"].str.replace(",","").astype(float)
df['SalaryUSD']= pd.to_numeric(df["SalaryUSD"])
df.head()

# Country Selection Options List

In [ ]:
countries = df[['Country']].groupby(['Country']).count()

# Load country list as option for multi select dropdown select
optionsCountry =[{'label': "Select All", 'value': -1}]
for i in range(len(countries.index)):
    optionsCountry.append({'label': countries.index[i], 'value': countries.index[i]})

countries

### Job Title and Gender selection lists

In [ ]:
jobTitle=df['JobTitle'].unique()

genderList=df['Gender'].unique()

# Bootstrap Dashboard App

In [14]:
import dash
import dash_bootstrap_components as dbc
#from dash.dependencies import Input, Output
from dash import Input, Output, dcc, html

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2("Sidebar", className="display-4"),
        html.Hr(),
        dbc.Nav(
            [
                dbc.NavLink("Home", href="/", active="exact"),
                dbc.NavLink("Page 1", href="/page-1", active="exact"),
                dbc.NavLink("Job Experince", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),

        html.Hr(),
        
        html.Label('Group By'),
        html.Br(),
        dcc.Dropdown(
            id="groupby",
            options= [{'label': 'Country', 'value': 'Country'},
                      {'label': 'Survey Year', 'value': 'Survey Year'},
                      {'label': 'Employment Sector', 'value': 'EmploymentSector'},
                      {'label': 'Employment Status', 'value': 'EmploymentStatus'},
                      {'label': 'Manage Staff', 'value': 'ManageStaff'},
                      {'label': 'Gender', 'value': 'Gender'},
                      {'label': 'Career Plans This Year', 'value': 'CareerPlansThisYear'}],
            value= 'Country',
        ),

        html.Br(),
        html.Label('Operation'),
        html.Br(),
        dcc.Dropdown(
            id="operation",
            options= [{'label': 'sum()', 'value': 'sum'},
                      {'label': 'count()', 'value': 'count'}],
            value= 'sum',
        ),

        html.Br(),
        html.Label('Country'),
        html.Br(),
        dcc.Dropdown(
            id="country",
            options= optionsCountry,
            value= -1,
            multi= True
        ),

        html.Br(),
        html.P('Plot Type', style={
        }),
        dbc.Card([dbc.RadioItems(
            id='plot_radio_items',
            value="1",
            options=[{
                'label': 'Bar',
                'value': '1'
            },
                {
                    'label': 'Line',
                    'value': '2'
                },
              
            ],
            
        )]),
     
        

    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content])


@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return dcc.Graph(id="graphCountry")
    elif pathname == "/page-1":
        return html.P("This is the content of page 1. Asma!")
    elif pathname == "/page-2":
        ############# Job Experince ###############
        return  [
                html.H1('Experince years for a specific job title',
                        style={'textAlign':'center'}),
                        
                html.Br(),
                dcc.RadioItems(
                    id="gender-slider",
                    value="Female",
                    options=[
                            {"label": gender, "value": gender}
                            for gender in genderList],
                        
                ),
                html.Br(),
                dcc.Dropdown(id="slct_job",
                            options=[
                                {"label": job, "value": job} for job in jobTitle],
                            multi=False,
                            value="Manager",
                            ),
                
                

                dcc.Graph(id="Experince"),
                ]
            ############ End Job Experince ##############

    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

@app.callback(Output('graphCountry', 'figure'), [Input("country", "value"),Input("groupby", "value"),Input("operation", "value"),Input("plot_radio_items","value"),])
def update_figure(val_country, val_groupby, val_operation,plot_radio_items):
    data = df

    if (val_country != -1):
        data = data[data["Country"].isin(val_country)]

    if (val_operation == 'sum'):
        data = data.groupby([val_groupby]).sum()
    if (val_operation == 'count'):
        data = data.groupby([val_groupby]).count()

    data = data.reset_index()
    
    plot =0 
    if (plot_radio_items == '1' ):
        plot = px.bar(
                data,
                x= val_groupby,
                y= "SalaryUSD"
            )
    elif (plot_radio_items =='2' ):
        plot = px.line(data, x=val_groupby, y="SalaryUSD", markers=True)
    return plot

############# Job Experince ###############
@app.callback(
    Output('Experince', 'figure'),

    [Input("gender-slider", "value"),
    Input("slct_job", "value"),]
)  
def update_Experince(slctdGender, slctdJob):
    data = df.copy()

    data = data[data["Gender"] == slctdGender]
    data = data[data["JobTitle"] == slctdJob]

    fig = px.box(data, y="YearsWithThisTypeOfJob")

    return fig
############ End Job Experince ##############
    

if __name__ == "__main__":
    app.run_server(port=8888)
